In [21]:
import os
import leafmap
from samgeo import SamGeo, tms_to_geotiff
from pyproj import Transformer
import matplotlib
matplotlib.use('nbagg')  # Use the interactive backend for widgets


In [23]:
import leafmap
from pyproj import Transformer

# Define the bounding box in EPSG:3857 (Web Mercator)
bbox_3857 = [-13610577.729, 4560823.588, -13609041.520, 4561655.600]

# Transform bbox to EPSG:4326 (latitude/longitude)
transformer = Transformer.from_crs("EPSG:3857", "EPSG:4326", always_xy=True)
min_lon, min_lat = transformer.transform(bbox_3857[0], bbox_3857[1])
max_lon, max_lat = transformer.transform(bbox_3857[2], bbox_3857[3])
bbox_4326 = [min_lon, min_lat, max_lon, max_lat]

print("Transformed bbox (EPSG:4326):", bbox_4326)

# Initialize the map centered around the transformed bounding box
m = leafmap.Map(center=[(min_lat + max_lat) / 2, (min_lon + max_lon) / 2], zoom=17, height="800px")

# Restrict the map to the bounding box of the image
m.fit_bounds([[min_lat, min_lon], [max_lat, max_lon]])

# Add the image as a raster layer
image = "ScanAI/SamGeo/Rensvik.tif"  # Path to your image
m.add_raster(image, layer_name="Rensvik Image")

# Optionally remove extra layers (e.g., basemap) to only display the image
m.layers = [layer for layer in m.layers if layer.name == "Rensvik Image"]

# Display the map
m


Transformed bbox (EPSG:4326): [-122.26589999657467, 37.868199999716396, -122.25209999633164, 37.874099996996726]


Map(center=[63.0966015, 7.8194035], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

In [24]:
# Use your custom image
image = "ScanAI/SamGeo/Rensvik.tif"
print("File exists:", os.path.exists(image))
print("Absolute path:", os.path.abspath(image))

# Add your raster image to the map
m.add_raster(image, layer_name="Rensvik Image")
#m


File exists: True
Absolute path: /home/siverteh/ScanAI/SamGeo/Rensvik.tif


In [25]:
sam = SamGeo(
    model_type="vit_h",
    automatic=False,
    sam_kwargs=None,
)

In [26]:
sam.set_image(image)

In [30]:
m = sam.show_map()
m

Map(center=[63.0966015, 7.8194035], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

No such comm: bb2c686616ff4062b40ea0642a49b74d


In [6]:
import os
from samgeo import tms_to_geotiff
from localtileserver import get_leaflet_tile_layer, TileClient
from ipyleaflet import Map
from ipywidgets import Layout
from pyproj import Transformer

# Define the bounding box in EPSG:3857 (Web Mercator)
bbox_3857 = [-13610577.729, 4560823.588, -13609041.520, 4561655.600]
transformer = Transformer.from_crs("EPSG:3857", "EPSG:4326", always_xy=True)
min_lon, min_lat = transformer.transform(bbox_3857[0], bbox_3857[1])
max_lon, max_lat = transformer.transform(bbox_3857[2], bbox_3857[3])
bbox_4326 = [min_lon, min_lat, max_lon, max_lat]
print("Transformed bbox (EPSG:4326):", bbox_4326)

# Use tms_to_geotiff to generate a GeoTIFF
image_path = "ScanAI/SamGeo/Rensvik_visualization.tif"
tms_to_geotiff(output=image_path, bbox=bbox_4326, zoom=16, source="Satellite", overwrite=True)

# Check if the GeoTIFF file was created
if not os.path.exists(image_path):
    raise FileNotFoundError(f"Generated TIFF not found: {image_path}")
print("GeoTIFF created:", os.path.abspath(image_path))

# Serve the GeoTIFF as a tile layer using TileClient
tile_client = TileClient(image_path)
tile_layer = get_leaflet_tile_layer(tile_client)

# Create a map centered on the image
center_lat = (min_lat + max_lat) / 2
center_lon = (min_lon + max_lon) / 2

m = Map(
    center=(center_lat, center_lon),
    zoom=16,
    layout=Layout(width="800px", height="600px"),
    scroll_wheel_zoom=True,
)
m.add_layer(tile_layer)

# Display the map
m


Transformed bbox (EPSG:4326): [-122.26589999657467, 37.868199999716396, -122.25209999633164, 37.874099996996726]
Downloaded image 1/6
Downloaded image 2/6
Downloaded image 3/6
Downloaded image 4/6
Downloaded image 5/6
Downloaded image 6/6
Saving GeoTIFF. Please wait...
Image saved to ScanAI/SamGeo/Rensvik_visualization.tif
GeoTIFF created: /home/siverteh/ScanAI/SamGeo/Rensvik_visualization.tif


HTTPError: 500 Server Error: INTERNAL SERVER ERROR for url: http://127.0.0.1:43281/api/metadata?&filename=%2Fhome%2Fsiverteh%2FScanAI%2FSamGeo%2FRensvik_visualization.tif

NameError: name 'gdalinfo' is not defined